In [1]:
import pandas as pd
from pathlib import Path
OUT_DIR = Path('join_gpt_cebab')

In [18]:
from datasets import load_dataset
dataset = load_dataset("CEBaB/CEBaB")

C:\Users\MatanAvitan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train_observational split: 100%|█████████████████████████████| 1755/1755 [00:00<00:00, 292672.40 examples/s]


In [34]:
cebab_dev = dataset['validation'].to_pandas()
cebab_dev[cebab_dev['is_original']==True].shape

(272, 24)

In [22]:
cebab_train = dataset['train_inclusive'].to_pandas().rename({'description': 'text'}, axis=1)
cebab_train

,id,original_id,edit_id,is_original,edit_goal,edit_type,edit_worker,text,review_majority,review_label_distribution,...,noise_aspect_majority,food_aspect_label_distribution,ambiance_aspect_label_distribution,service_aspect_label_distribution,noise_aspect_label_distribution,food_aspect_validation_workers,ambiance_aspect_validation_workers,service_aspect_validation_workers,noise_aspect_validation_workers,opentable_metadata
0,0,0,0,True,None,None,None,"Overbooked and didnot honor reservation time,p...",1,"{'1': 4, '2': 1}",...,unknown,,,{'Negative': 5},"{'unknown': 4, 'Negative': 1}",,,"{'w148': 'Negative', 'w120': 'Negative', 'w83'...","{'w27': 'unknown', 'w23': 'unknown', 'w81': 'N...","{'restaurant_id': 6513, 'restaurant_name': ""Mo..."
1,1,0,1,False,Negative,noise,w82,"Overbooked and didnot honor reservation time,p...",no majority,"{'1': 2, '2': 2, '3': 1}",...,Negative,,,{'Negative': 5},{'Negative': 5},,,"{'w148': 'Negative', 'w120': 'Negative', 'w83'...","{'w16': 'Negative', 'w63': 'Negative', 'w36': ...","{'restaurant_id': 6513, 'restaurant_name': ""Mo..."
2,2,0,2,False,Positive,service,w59,Overbooked but honored our reservations and we...,no majority,"{'3': 2, '2': 2, '5': 1}",...,unknown,,,"{'Positive': 3, 'unknown': 2}","{'unknown': 4, 'Negative': 1}",,,"{'w10': 'Positive', 'w162': 'unknown', 'w7': '...","{'w27': 'unknown', 'w23': 'unknown', 'w81': 'N...","{'restaurant_id': 6513, 'restaurant_name': ""Mo..."
3,3,0,3,False,Positive,noise,w38,"Overbooked, but still pleasant as far as the n...",3,"{'3': 4, '4': 1}",...,Positive,,,{'Negative': 5},"{'Positive': 4, 'Negative': 1}",,,"{'w148': 'Negative', 'w120': 'Negative', 'w83'...","{'w170': 'Negative', 'w17': 'Positive', 'w35':...","{'restaurant_id': 6513, 'restaurant_name': ""Mo..."
4,1000000,1,0,True,None,None,None,I was really disappointed with the meal. The g...,1,"{'1': 4, '2': 1}",...,,{'Negative': 5},,{'Negative': 5},,"{'w147': 'Negative', 'w47': 'Negative', 'w83':...",,"{'w92': 'Negative', 'w107': 'Negative', 'w152'...",,"{'restaurant_id': 3297, 'restaurant_name': 'No..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11723,1754000001,1754,1,False,Negative,food,w4,"A combination of great service, innovative but...",3,"{'3': 3, '4': 1, '5': 1}",...,,"{'Negative': 4, 'unknown': 1}","{'unknown': 4, 'Positive': 1}",{'Positive': 5},,"{'w21': 'Negative', 'w73': 'unknown', 'w62': '...","{'w18': 'Positive', 'w13': 'unknown', 'w27': '...","{'w16': 'Positive', 'w17': 'Positive', 'w50': ...",,"{'restaurant_id': 43678, 'restaurant_name': 'P..."
11724,1754000002,1754,2,False,Negative,ambiance,w79,"A combination of great service, innovative but...",3,"{'3': 3, '2': 2}",...,,{'Positive': 5},{'unknown': 5},{'Positive': 5},,"{'w18': 'Positive', 'w23': 'Positive', 'w14': ...","{'w17': 'unknown', 'w35': 'unknown', 'w70': 'u...","{'w16': 'Positive', 'w17': 'Positive', 'w50': ...",,"{'restaurant_id': 43678, 'restaurant_name': 'P..."
11725,1754000003,1754,3,False,Negative,service,w34,It is too bad the service is so bad at this pl...,3,"{'3': 3, '2': 1, '1': 1}",...,,{'Positive': 5},"{'unknown': 4, 'Positive': 1}",{'Negative': 5},,"{'w18': 'Positive', 'w23': 'Positive', 'w14': ...","{'w18': 'Positive', 'w13': 'unknown', 'w27': '...","{'w61': 'Negative', 'w57': 'Negative', 'w84': ...",,"{'restaurant_id': 43678, 'restaurant_name': 'P..."
11726,1754000004,1754,4,False,unknown,food,w17,A combination of great service and moderate pr...,5,"{'5': 3, '4': 2}",...,,{'unknown': 5},"{'unknown': 4, 'Positive': 1}",{'Positive': 5},,"{'w199': 'unknown', 'w2': 'unknown', 'w35': 'u...","{'w18': 'Positive', 'w13': 'unknown', 'w27': '...","{'w16': 'Positive', 'w17': 'Positive', 'w50': ...",,"{'restaurant_id': 43678, 'restaurant_name': 'P..."


In [28]:
cebab_gpt_mimic_food = pd.read_csv('cebab_gpt_mimic_food.csv')
cebab_gpt_mimic_service = pd.read_csv('cebab_gpt_no_int.csv')

In [31]:
cebab_gpt_mimic_food

,labels_food,labels_service,text,transformed_hard_text
0,0,0,Service was slow and inattentive. The steak an...,Service was slow and inattentive. The steak an...
1,0,0,Food at the special Mother's Day brunch was ex...,Food at the special Mother's Day brunch was ex...
2,0,1,The food was disappointing and our waiter was ...,"The food was exceptional, our waiter was amazi..."
3,0,0,Disappointing service for all 4 guests. Food w...,Disappointing service. Food was underwhelming ...
4,0,1,Decent food and VERY relaxing service experien...,Decent food and VERY relaxing service experien...
...,...,...,...,...
2610,1,0,Food for my birthday celebration was below exp...,Food for my birthday celebration was disappoin...
2611,1,1,"Despite the food not meeting expectations, the...","Despite not meeting expectations, the food exc..."
2612,1,0,"Service nonexistent, but the food was exceptio...","Service nonexistent, but the food was exceptio..."
2613,1,1,Food was absolutely delicious - exactly what y...,Service was impeccable. Food was incredibly ta...


In [30]:
cebab_gpt_mimic_food_join = pd.merge(cebab_train[['text']],
                               cebab_gpt_mimic_food.rename({'transformed_hard_text': 'intervention_hard_text'}, axis=1),
                               how='inner', on=['text'])
cebab_gpt_mimic_food_join

,text,labels_food,labels_service,intervention_hard_text
0,We enjoyed the ambiance and the restaurant but...,0,0,We enjoyed the ambiance in the restaurant and ...
1,I don't mind paying the higher food prices for...,0,0,I don't mind paying the higher food prices for...
2,This restaurant has obviously changed manageme...,0,1,This restaurant has obviously changed manageme...
3,This restaurant has obviously changed manageme...,0,0,This restaurant has obviously changed manageme...
4,Food was mediocre.,0,0,Food was exceptional. The mediocre food was ve...
5,"Terrible service, terrible location, terrible ...",0,0,"Terrible service, terrible location, great foo..."
6,Our waitress was fantastic but the food was me...,0,0,Our waitress was fantastic at best but the foo...


In [13]:
cebab_gpt_mimic_food_join.drop_duplicates(['text'])

,text,transformed_hard_text,labels_food,labels_service,intervention_hard_text
0,Despite the special menu item being unavailabl...,Despite the special menu item being unavailabl...,1,1,Despite the special menu item being unavailabl...
1,"Service was impeccable, however, the new menu ...","Service was impeccable, however, the new menu ...",1,1,"Service was impeccable, however, the new menu ..."
2,Service was slow and inattentive. The steak an...,Service was slow and inattentive. The steak an...,0,0,Service was slow and inattentive. The steak an...
3,Food at the special Mother's Day brunch was ex...,Food at the special Mother's Day brunch was ex...,0,0,Food at the special Mother's Day brunch was ex...
4,The food was disappointing and our waiter was ...,The food was disappointing and our waiter was ...,0,1,"The food was exceptional, our waiter was amazi..."
...,...,...,...,...,...
2614,"Despite the dry and flavorless fish, the servi...","Despite the dry and flavorless fish, the servi...",0,1,"Despite the dry and flavorless fish, the food,..."
2615,"Service nonexistent, but the food was exceptio...","Service nonexistent, but the food was exceptio...",1,0,"Service nonexistent, but the food was exceptio..."
2616,"Food was disappointing, service was exceptiona...","Food was disappointing, service was exceptiona...",0,1,"Food was delicious, service was exceptional, a..."
2617,Food was absolutely delicious - exactly what y...,Food was absolutely delicious - exactly what y...,1,1,Service was impeccable. Food was incredibly ta...


In [25]:
bios_mimic_plus_mal_to_fem_join.to_csv(OUT_DIR / 'bios_mimic_plus_mal_to_fem_join.csv')